In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv",index_col="Id")
test = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv",index_col="Id")
sample_sub = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv",index_col="Id")
greek = pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/greeks.csv",index_col="Id")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 617 entries, 000ff2bfdfe9 to ffcca4ded3bb
Data columns (total 57 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AB      617 non-null    float64
 1   AF      617 non-null    float64
 2   AH      617 non-null    float64
 3   AM      617 non-null    float64
 4   AR      617 non-null    float64
 5   AX      617 non-null    float64
 6   AY      617 non-null    float64
 7   AZ      617 non-null    float64
 8   BC      617 non-null    float64
 9   BD      617 non-null    float64
 10  BN      617 non-null    float64
 11  BP      617 non-null    float64
 12  BQ      557 non-null    float64
 13  BR      617 non-null    float64
 14  BZ      617 non-null    float64
 15  CB      615 non-null    float64
 16  CC      614 non-null    float64
 17  CD      617 non-null    float64
 18  CF      617 non-null    float64
 19  CH      617 non-null    float64
 20  CL      617 non-null    float64
 21  CR      617 non-null    

In [4]:
train['EJ'] = train['EJ'].replace({'A': 0, 'B': 1})
test['EJ'] = test['EJ'].replace({'A': 0, 'B': 1})

In [5]:
train = train.dropna(axis = 0)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 548 entries, 000ff2bfdfe9 to ffcca4ded3bb
Data columns (total 57 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AB      548 non-null    float64
 1   AF      548 non-null    float64
 2   AH      548 non-null    float64
 3   AM      548 non-null    float64
 4   AR      548 non-null    float64
 5   AX      548 non-null    float64
 6   AY      548 non-null    float64
 7   AZ      548 non-null    float64
 8   BC      548 non-null    float64
 9   BD      548 non-null    float64
 10  BN      548 non-null    float64
 11  BP      548 non-null    float64
 12  BQ      548 non-null    float64
 13  BR      548 non-null    float64
 14  BZ      548 non-null    float64
 15  CB      548 non-null    float64
 16  CC      548 non-null    float64
 17  CD      548 non-null    float64
 18  CF      548 non-null    float64
 19  CH      548 non-null    float64
 20  CL      548 non-null    float64
 21  CR      548 non-null    

In [7]:
from sklearn import preprocessing
x = preprocessing.normalize(train.drop("Class",axis =1))
test = preprocessing.normalize(test)

In [8]:
from sklearn.model_selection import train_test_split
x, x_cv, y, y_cv = train_test_split(x, train["Class"], test_size=0.33, random_state=42)

Logistic regression

In [9]:
from sklearn.linear_model import LogisticRegression
model_1 = LogisticRegression(random_state=42,max_iter=1000).fit(x, y)

In [10]:
from sklearn.ensemble import RandomForestClassifier 
model_2 = RandomForestClassifier(n_estimators = 100) 
model_2 .fit(x,y)

RandomForestClassifier()

In [11]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [12]:
sol_1 = model_1.predict(x_cv)
sol_2 = model_2.predict(x_cv)
sol_3 = model.predict(x_cv)

In [13]:
from sklearn.metrics import accuracy_score
print(accuracy_score(sol_1, y_cv, normalize=True))
print(accuracy_score(sol_3, y_cv, normalize=True))
print(accuracy_score(sol_2, y_cv, normalize=True))

0.8176795580110497
0.8784530386740331
0.8729281767955801


In [14]:
submit=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv",index_col="Id")
submit=submit.drop(submit[:],axis =1)

In [15]:

submit["class_0"]=1-model_2.predict(test)
submit["class_1"]=model_2.predict(test)
submit.to_csv('submission.csv',index=False)